In [5]:
import re
import bs4
import queue
import json
import sys
import csv
import urllib
import os.path
import datetime
from os.path import join as pjoin
import time
import zipfile
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import seaborn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDiA

In [8]:
meade_df = pd.read_csv("meade.csv",encoding='UTF-16')
#Outcome = 1 if amlo, 0 ioc
meade_df["outcome"] = 0
del meade_df['date']


anaya_df = pd.read_csv("anaya_bis.csv",encoding='UTF-16')
anaya_df.columns = ['speech']
anaya_df["outcome"] = 0

amlo_df = pd.read_csv("amlo_bis.csv",encoding='UTF-16')
del amlo_df["date"]
amlo_df.columns = ['speech']
amlo_df["outcome"] = 1

frames = [meade_df, anaya_df, amlo_df]

#Merging
speeches = pd.concat(frames)
#speeches.to_csv('speeches.csv',  encoding='UTF-16', index=False)
speeches.head()

,speech,outcome
0,José Antonio Meade: ¡Buenos días Chalco!¿Cómo ...,0
1,"José Antonio Meade: ¡Muy buenas tardes, Fresni...",0
2,"Pregunta: Me da mucho gusto saludarte.Meade, m...",0
3,José Antonio Meade: Les agradezco mucho a todo...,0
4,José Antonio Meade: Muy buenas tardes.¡Buenas ...,0


In [9]:
#PCA analysis
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=speeches.speech).toarray()
tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape

(495, 29845)

In [10]:
speeches.outcome.sum()

191

In [11]:

pca = PCA(n_components=10)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, 
                                 columns=columns)
pca_topic_vectors.round(3).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,0.160,0.755,-0.121,-0.002,-0.052,-0.011,-0.005,-0.004,-0.029,-0.003
1,0.164,0.690,-0.101,0.002,-0.070,0.053,-0.022,0.054,0.061,0.025
2,-0.129,0.290,0.141,-0.081,-0.078,-0.016,0.030,-0.035,0.010,-0.022
3,-0.234,-0.008,0.176,-0.000,-0.031,0.080,0.001,-0.028,0.018,0.005
4,0.169,0.741,-0.076,-0.026,-0.057,0.023,-0.006,0.039,0.004,0.010


In [12]:
svd = TruncatedSVD(n_components=10, n_iter=300)  
svd_topic_vectors = svd.fit_transform(tfidf_docs)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, 
                                 columns=columns)
svd_topic_vectors.round(2).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,0.16,0.76,-0.12,-0.00,-0.05,-0.01,-0.00,-0.00,-0.02,-0.01
1,0.16,0.69,-0.10,0.00,-0.07,0.05,-0.02,0.06,0.05,0.03
2,-0.13,0.29,0.14,-0.08,-0.08,-0.02,0.03,-0.04,0.02,-0.01
3,-0.23,-0.01,0.18,-0.00,-0.03,0.08,0.00,-0.03,0.02,0.00
4,0.17,0.74,-0.08,-0.03,-0.06,0.02,-0.01,0.04,0.00,0.01
5,-0.19,-0.08,0.21,-0.01,-0.08,0.06,0.02,-0.04,0.00,-0.01


In [13]:
svd_topic_vectors = (svd_topic_vectors.T / 
                     np.linalg.norm(svd_topic_vectors, axis=1)).T
#Meade
svd_topic_vectors.iloc[:6].dot(svd_topic_vectors.iloc[:6].T).round(3)

,0,1,2,3,4,5
0,1.000,0.986,0.641,-0.268,0.994,-0.456
1,0.986,1.000,0.624,-0.251,0.994,-0.441
2,0.641,0.624,1.000,0.486,0.661,0.338
3,-0.268,-0.251,0.486,1.000,-0.238,0.937
4,0.994,0.994,0.661,-0.238,1.000,-0.423
5,-0.456,-0.441,0.338,0.937,-0.423,1.000


In [14]:
#AMLO
svd_topic_vectors.iloc[488:].dot(svd_topic_vectors.iloc[488:].T).round(3)

,488,489,490,491,492,493,494
488,1.000,0.749,0.519,0.811,0.748,0.728,0.792
489,0.749,1.000,0.626,0.982,0.923,0.750,0.969
490,0.519,0.626,1.000,0.678,0.621,0.513,0.585
491,0.811,0.982,0.678,1.000,0.942,0.791,0.970
492,0.748,0.923,0.621,0.942,1.000,0.789,0.926
493,0.728,0.750,0.513,0.791,0.789,1.000,0.775
494,0.792,0.969,0.585,0.970,0.926,0.775,1.000


In [15]:
#Anaya
svd_topic_vectors.iloc[200:206].dot(svd_topic_vectors.iloc[200:206].T).round(3)

,200,201,202,203,204,205
200,1.000,0.624,0.604,0.585,0.582,0.496
201,0.624,1.000,0.938,0.905,0.934,0.869
202,0.604,0.938,1.000,0.934,0.985,0.926
203,0.585,0.905,0.934,1.000,0.924,0.872
204,0.582,0.934,0.985,0.924,1.000,0.939
205,0.496,0.869,0.926,0.872,0.939,1.000


# PCA

In [16]:
pca10_topic_vectors = pca.fit_transform(tfidf_docs)


X_train, X_test, y_train, y_test = train_test_split(pca10_topic_vectors,
                                                    speeches.outcome.astype(int),
                                                    test_size=0.5, 
                                                    random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
speeches['pca10_outcome'] = lda.predict(pca10_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

0.988

# LDiA

In [19]:
# Average number of words per speech
texts = speeches.speech
sum([len(t.split()) for t in texts]) * 1. / len(texts)

1391.8727272727272

In [21]:
counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=speeches.speech).toarray())
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms

In [23]:

ldia = LDiA(n_components=10, learning_method='batch')
ldia = ldia.fit(bow_docs)

components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)
components.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
!,0.27,0.1,0.1,2969.08,11.54,0.1,0.1,3.51,0.1,0.1
#actúamx,0.10,0.1,0.1,0.10,1.10,0.1,0.1,0.10,0.1,0.1
#dddddd,0.10,0.1,0.1,0.10,1.10,0.1,0.1,0.10,0.1,0.1
#ddddddciudad,0.10,0.1,0.1,0.10,1.10,0.1,0.1,0.10,0.1,0.1
#noesnormal,0.10,0.1,0.1,11.10,0.10,0.1,0.1,0.10,0.1,0.1


In [40]:
components.topic1.sort_values(ascending=False)[:10]

,      20077.886880
de     12537.946700
que    11367.117577
la      9074.640189
a       7683.519103
el      6852.762251
y       6161.894895
en      5511.226675
se      5384.429944
los     5006.229569
Name: topic1, dtype: float64

In [43]:
ldia10_topic_vectors = ldia.transform(bow_docs)
ldia10_topic_vectors = pd.DataFrame(ldia10_topic_vectors, columns=columns)
ldia10_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,0.0,0.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.87,0.13,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.58,0.42,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.00,1.00,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.00,0.00,0.0,0.0,0.0,0.0,0.0


In [45]:
X_train, X_test, y_train, y_test = train_test_split(ldia10_topic_vectors, speeches.outcome,
    test_size=0.5, random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
speeches['ldia10_outcome'] = lda.predict(ldia10_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.964

In [48]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_docs, speeches.outcome, test_size=0.5, random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
round(float(lda.score(X_train, y_train)), 3)

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.619

In [49]:
round(float(lda.score(X_test, y_test)), 3)

0.468

In [50]:
ldia32 = LDiA(n_components=32, learning_method='batch')
ldia32 = ldia32.fit(bow_docs)
ldia32.components_.shape

(32, 29845)

In [51]:
ldia32_topic_vectors = ldia32.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(ldia32.n_components)]
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors, columns=columns32)
ldia32_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31
0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00,0.0,0.12,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.04,0.0,0.10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.06,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.0


In [53]:
X_train, X_test, y_train, y_test = train_test_split(ldia32_topic_vectors, speeches.outcome,
    test_size=0.5, random_state=271828)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
speeches['ldia32_outcome'] = lda.predict(ldia32_topic_vectors)
X_train.shape

/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(247, 32)

In [54]:
round(float(lda.score(X_train, y_train)), 3)

0.988

In [55]:
round(float(lda.score(X_test, y_test)), 3)

0.988